In [1]:
import pixiedust

if sc.version.startswith('1.6.'):  # Spark 1.6
    pixiedust.installPackage("graphframes:graphframes:0.5.0-spark1.6-s_2.11")
elif sc.version.startswith('2.'):  # Spark 2.1, 2.0
    pixiedust.installPackage("graphframes:graphframes:0.5.0-spark2.1-s_2.11")


pixiedust.installPackage("com.typesafe.scala-logging:scala-logging-api_2.11:2.1.2")
pixiedust.installPackage("com.typesafe.scala-logging:scala-logging-slf4j_2.11:2.1.2")

print("done")

Pixiedust database opened successfully


Package already installed: graphframes:graphframes:0.5.0-spark2.1-s_2.11
Package already installed: com.typesafe.scala-logging:scala-logging-api_2.11:2.1.2
Package already installed: com.typesafe.scala-logging:scala-logging-slf4j_2.11:2.1.2
done


In [2]:
spark.sparkContext.getConf().getAll()

[('spark.master', 'spark://128.235.40.174:7077'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '10G'),
 ('spark.driver.extraClassPath', '/home/hao/pixiedust/data/libs/*'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.jars', 'file:/home/hao/pixiedust/bin/cloudant-spark-v2.0.0-185.jar'),
 ('spark.app.id', 'app-20171201120134-0016'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.driver.port', '41409'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.host', '128.235.40.174'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.deployMode', 'client'),
 ('spark.executor.memory', '4G')]

In [ ]:

spark.conf.set("spark.submit.deployMode", "client")
spark.conf.set("spark.executor.memory", "2G")
sc = spark.sparkContext
sc.getConf().getAll()

In [3]:
sqlContext=SQLContext(sc)
# read union parquet
# df_union = sqlContext.read.parquet("./data/df_union.parquet")
df_union = sqlContext.read.parquet("hdfs://128.235.40.174:9000/data/df_union.parquet")
pixiedust.display(df_union)

# read hier parquet
# df_hier = sqlContext.read.parquet("./data/df_hier.parquet")
df_hier = sqlContext.read.parquet("hdfs://128.235.40.174:9000/data/df_hier.parquet")
pixiedust.display(df_hier)

src,dst,ont
http://bioontology.org/projects/ontologies/birnlex#birnlex_7346,http://bioontology.org/projects/ontologies/birnlex#birnlex_471,birnlex.owl
http://purl.obolibrary.org/obo/chebi_103512,http://purl.obolibrary.org/obo/chebi_24995,chebi.obo
http://purl.obolibrary.org/obo/doid_13450,http://purl.obolibrary.org/obo/doid_0050292,doid.obo
http://purl.obolibrary.org/obo/chebi_16162,http://purl.obolibrary.org/obo/chebi_36084,chebi.obo
http://purl.obolibrary.org/obo/dron_00745389,http://purl.obolibrary.org/obo/dron_00000027,dron.owl
http://purl.obolibrary.org/obo/dron_00011383,http://purl.obolibrary.org/obo/obi_0000047,dron.owl
http://purl.obolibrary.org/obo/caro_0000068,http://purl.obolibrary.org/obo/caro_0000073,aeo.obo
http://purl.obolibrary.org/obo/chebi_70769,http://purl.obolibrary.org/obo/chebi_84194,chebi.obo
http://purl.obolibrary.org/obo/dron_00508557,http://purl.obolibrary.org/obo/dron_00000027,dron.owl
http://purl.obolibrary.org/obo/dron_00129981,http://purl.obolibrary.org/obo/dron_00000027,dron.owl


In [ ]:
# remove duplicated label
import re
def isIRIEqualLabel(iri, label):
    iri = iri.split('/')[-1].replace('_',' ')
    return label ==iri 


# df_class_labels
# df_class_hier
# df_tax_areas
# df_tax_areas_concepts
# df_tax_areas_pareas

# work with df_union

label = 'information content entity'
iri= 'http://purl.obolibrary.org/obo/iao_0000030'

# get iri
def getIRI(label):
    result = []
    iris = df_union.filter(df_union.label==label).select('id','label').distinct().collect()
    for row in iris:
        if not isIRIEqualLabel(row["id"], row['label']):
            result.append(row["id"])
    return result

print(getIRI(label))

# get label
def getLabel(iri):
    result =[]
    labels = df_union.filter(df_union.id==iri).select('id','label').distinct().collect()
    for row in labels:
        if not isIRIEqualLabel(row["id"], row['label']):
            result.append(row["label"])
    if not result:
        result.append(labels[0]['label'])
    return result

print(getLabel(iri))

# get ont
def getOnts(iri):
    result =[]
    ont_names = df_union.filter(df_union.id==iri).select('ont').distinct().collect()
    for row in ont_names:
        result.append(row["ont"])
    return result

print(getOnts(iri))

# get area info DF
def getAreaInfoDF(iri):
    return df_union.filter(df_union.id==iri).distinct()

# get area id from areaInfoDF
def getAreaDetail(areaInfoDF, ont, tax_type):
    return areaInfoDF.filter((areaInfoDF.ont==ont) & (areaInfoDF.type== tax_type)).first()

def getAreaID(areaDetailDF):
    if areaDetailDF:
        return areaDetailDF['area_id']
    else:
        return 'Not applicable'

def getAreaName(areaDetailDF):
    if areaDetailDF:
        return areaDetailDF['area_name']
    else:
        return 'Not applicable'

def getAreaLevel(areaDetailDF):
    if areaDetailDF:
        return areaDetailDF['area_level']
    else:
        return 'Not applicable'

    
  

In [ ]:
import time

start_time = time.time()  
    
    
iri_list=['http://purl.obolibrary.org/obo/chebi_23014',\
'http://purl.obolibrary.org/obo/chebi_38670',\
'http://purl.obolibrary.org/obo/chebi_17102',\
'http://purl.obolibrary.org/obo/chebi_76836',\
'http://purl.obolibrary.org/obo/uberon_0000916',\
'http://purl.obolibrary.org/obo/cdao_0000127',\
'http://purl.obolibrary.org/obo/bto_0003205',\
'http://www.ifomis.org/bfo/1.1/snap#role']    

for iri in iri_list: 
    areaInfoDF = getAreaInfoDF(iri)
    label = getLabel(iri)
    print(label)
    ontology = getOnts(iri)[0]
    areaDetailDF=getAreaDetail(areaInfoDF, ontology, 'op_restriction')
    getAreaID(areaDetailDF)
    getAreaName(areaDetailDF)
    getAreaLevel(areaDetailDF)
# print('done')

print(time.time() - start_time, "seconds")


# get area name

# get tax type

# get parea root id

# get area level

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Button, Layout

ontList =[]
iri =None

# search by iri or label or other

searchType=widgets.ToggleButtons(
    options=['IRI', 'Label', 'Other'],
    description='Search by:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Type in IRI in format of URL', 'Type in concept name', 'Feel Lucky'],
#     icons=['check'] * 3
)
display(searchType)

# search box
searchbox = widgets.Text(value='Search concept here', disabled=False)
display(searchbox)

# search button
searchButton = Button(description='Search')
searchButton.style.button_color = 'lightgreen'
display(searchButton)

def on_searchButton_clicked(b):
    global iri
    ont_dropdown.layout.visibility = 'hidden'
    type_dropdown.layout.visibility = 'hidden'
    conLabel =''
#     print("Button clicked.")
    if searchbox.value:
        text = searchbox.value
        if searchType.value == "IRI":
            iri = text
            conLabel = getLabel(text)[0]
            ontList = getOnts(text)
        elif searchType.value == "Label":
            iri = getIRI(text)
            conLabel = text
            ontList = getOnts(iri[0])
        else:
            ontList = [] 
        if ontList:
            ont_dropdown.options = ontList
            ont_dropdown.layout.visibility = 'visible'
            type_dropdown.layout.visibility = 'visible'
    conceptLabel.value = conLabel
    
searchButton.on_click(on_searchButton_clicked)

# label

style = {'description_width': 'initial'}

conceptLabel = widgets.Text(
    value='',
    placeholder='Concept Label',
    description='The Concept Label is: ',
    layout=Layout(width='70%'),
    disabled=True,
    style=style
)

areaIDLabel = widgets.Textarea(
    value='',
    placeholder='Area ID',
    description='The Area ID is: ',
    layout=Layout(width='70%'),
    disabled=True,
    style=style
)

areaNameLabel =  widgets.Textarea(
    value='',
    placeholder='Area Name',
    description='The Area Name is: ',
    layout=Layout(width='70%'),
    disabled=True,
    style=style
)
areaLevelLabel =  widgets.Text(
    value='',
    placeholder='Area Level',
    description='The Area Level is: ',
    disabled=True,
    style=style
)
display(conceptLabel)
display(areaIDLabel)
display(areaNameLabel)
display(areaLevelLabel)
# iri= 'http://purl.obolibrary.org/obo/iao_0000030'
# print(getOnts(iri)[0])
# choose ontology
ont_dropdown = widgets.Dropdown(
    options = ontList,
    value = None,
    description='Ontology:',

)
ont_dropdown.layout.visibility = 'hidden'
display(ont_dropdown)

def on_ont_dropdonw_change(change):
#     print(iri)
#     print(change['old'])
#     print(change['new'])
#     print(type_dropdown.value)
    areaInfoDF = getAreaInfoDF(iri)
    areaDetailDF=getAreaDetail(areaInfoDF, change['new'], type_dropdown.value)
    areaIDLabel.value = getAreaID(areaDetailDF).replace(", ", "\n")
    areaNameLabel.value =  getAreaName(areaDetailDF).replace(", ", "\n")
    areaLevelLabel.value =  str(getAreaLevel(areaDetailDF))
    
ont_dropdown.observe(on_ont_dropdonw_change, names= 'value')



# choose restriction type
type_dropdown=widgets.Dropdown(
    options=['op_restriction', 'op_domain', 'dp_restriction', 'dp_domain'],
    value='op_restriction',
    description='Tax Type:',
    disabled= False
)
type_dropdown.layout.visibility = 'hidden'
display(type_dropdown)

def on_type_dropdown_change(change):
#     print(iri)
#     print(change['old'])
#     print(change['new'])
#     print(ont_dropdown.value)
    areaInfoDF = getAreaInfoDF(iri)
    areaDetailDF=getAreaDetail(areaInfoDF, ont_dropdown.value, change['new'])
    areaIDLabel.value = getAreaID(areaDetailDF).replace(", ", "\n")
    areaNameLabel.value =  getAreaName(areaDetailDF).replace(", ", "\n")
    areaLevelLabel.value =  str(getAreaLevel(areaDetailDF))
    
type_dropdown.observe(on_type_dropdown_change, names= 'value')
    
# print(ont_dropdown.value)
# print(type_dropdown.value)

In [83]:
from graphframes import *
g = GraphFrame(df_union, df_hier)
g.inDegrees.show(20, False)
# g = GraphFrame(vertex, edges)

+------------------------------------------------------------------------------+--------+
|id                                                                            |inDegree|
+------------------------------------------------------------------------------+--------+
|http://purl.obolibrary.org/obo/chebi_23014                                    |10      |
|http://purl.obolibrary.org/obo/chebi_38670                                    |56      |
|http://purl.obolibrary.org/obo/dron_00065213                                  |1       |
|http://purl.obolibrary.org/obo/chebi_17102                                    |19      |
|http://purl.obolibrary.org/obo/chebi_76836                                    |4       |
|http://purl.obolibrary.org/obo/dron_00034855                                  |4       |
|http://purl.obolibrary.org/obo/dron_00043648                                  |7       |
|http://purl.obolibrary.org/obo/dron_00046391                                  |4       |
|http://pu

In [84]:
v2 = g.vertices.filter("area_name = 'has part, is conjugate base of' and parea_root_id = 'http://purl.obolibrary.org/obo/chebi_29067'")
e2 = g.edges.filter("ont = 'chebi.obo'")
g2 = GraphFrame(v2, e2)
print(g2.vertices.count())
print(g2.edges.count())
# pixiedust.display(g2)


1692
181470


In [86]:


# g2.vertices.printSchema()
# 
# iri ='http://purl.obolibrary.org/obo/ch0bi_29067'

# vert = g2.vertices.select('id').rdd.flatMap(lambda x: x).collect()
# print(vert[10])
# print(g2.vertices.where(g2.vertices.id==iri).collect())

# print(g2.edges.select('src').show(20,False))
# e3= g2.edges.filter( (g2.edges.src.isin(vert)==True) & (g2.edges.dst.isin(vert)==True) )
# e3=g2.edges.filter(g2.edges.src.isin(vert)==True) 
# print(e3.count())
# print(g2.edges.count())
def removeEdgesNotInVertices(g):
    e = g.edges
    v = g.vertices
    vert = v.select('id').rdd.flatMap(lambda x: x).collect()
    e2= e.filter((e.src.isin(vert)==True) & (e.dst.isin(vert)==True))
    return GraphFrame(v, e2)
    
    
    
g2 = removeEdgesNotInVertices(g2)
print(g2.vertices.count())
print(g2.edges.count())
# for row in g2.edges.collect():
#     print(row.src)

1692
2155


In [ ]:
sc.setCheckpointDir("hdfs://128.235.40.174:9000/RddCheckPoint")
result = g2.connectedComponents()
result.select("id", "component").orderBy("component").show()

In [ ]:
import time
from pyspark.sql.functions import *

start_time = time.time()

ranks = g2.pageRank(resetProbability=0.20, maxIter=3)

print(time.time() - start_time, "seconds")

In [ ]:
pixiedust.display(ranks.vertices.select("id","pagerank").orderBy(desc("pagerank")))

In [ ]:
results = g2.triangleCount()
results.select("id", "count").show()

In [ ]:
a= 'http://purl.obolibrary.org/obo/chebi_62943'
b= 'http://purl.obolibrary.org/obo/chebi_64012'
results = g2.shortestPaths(landmarks=[a, b])
results.select("id", "distances").show()

In [ ]:
a= 'http://purl.obolibrary.org/obo/chebi_33498'
# Run PageRank until convergence to tolerance "tol".
results = g2.pageRank(resetProbability=0.15, tol=0.01)
# Display resulting pageranks and final edge weights
# Note that the displayed pagerank may be truncated, e.g., missing the E notation.
# In Spark 1.5+, you can use show(truncate=False) to avoid truncation.
results.vertices.select("id", "pagerank").show()
results.edges.select("src", "dst", "weight").show()

# Run PageRank for a fixed number of iterations.
results2 = g2.pageRank(resetProbability=0.15, maxIter=5)

# Run PageRank personalized for vertex "a"
# results3 = g2.pageRank(resetProbability=0.15, maxIter=5, sourceId=a)

In [ ]:
import time
from pyspark.sql.functions import *

start_time = time.time()

paths = g.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[cd]->(d)")\
  .filter("b.area_level = 2 and c.ont = 'chebi.obo' and d.area_level >= a.area_level ")\
  .filter("a.id = 'http://purl.obolibrary.org/obo/chebi_62943'")

# display(paths)
paths.show()
print(time.time() - start_time, "seconds")



In [ ]:
paths = g.bfs(fromExpr="id = 'http://purl.obolibrary.org/obo/chebi_64012'", toExpr ="area_level = 5", edgeFilter = "ont = 'chebi.obo'")
paths.show(10, False)
# display(paths)

In [80]:
import igraph as ig
g = ig.Graph()

g.add_vertices(g2.vertices.rdd.map(lambda x:x.id).collect())
g.vs["label"] = g2.vertices.rdd.map(lambda x: x.label).collect()
g.vs["type"] = g2.vertices.rdd.map(lambda x: x.type).collect()
g.vs["area_id"] = g2.vertices.rdd.map(lambda x: x.area_id).collect()
g.vs["area_name"] = g2.vertices.rdd.map(lambda x: x.area_name).collect()
g.vs["area_level"] = g2.vertices.rdd.map(lambda x: x.area_level).collect()
g.vs["parea_root_id"] = g2.vertices.rdd.map(lambda x: x.parea_root_id).collect()

N=g.vcount()
print('total number of vertices imported: ' , N)
print(g.vs[0])
print(g.vs[0]['label'])
print(g.vs[1])
print(g.vs[1]['area_level'])

total number of vertices imported:  1692
igraph.Vertex(<igraph.Graph object at 0x7fd4dcb46e58>, 0, {'name': 'http://purl.obolibrary.org/obo/chebi_133943', 'area_id': 'http://purl.obolibrary.org/obo/bfo_0000051, http://purl.obolibrary.org/obo/chebi#is_conjugate_base_of', 'area_name': 'has part, is conjugate base of', 'type': 'op_restriction', 'area_level': 2, 'parea_root_id': 'http://purl.obolibrary.org/obo/chebi_29067', 'label': 'amoxicilloate'})
amoxicilloate
igraph.Vertex(<igraph.Graph object at 0x7fd4dcb46e58>, 1, {'name': 'http://purl.obolibrary.org/obo/chebi_57948', 'area_id': 'http://purl.obolibrary.org/obo/bfo_0000051, http://purl.obolibrary.org/obo/chebi#is_conjugate_base_of', 'area_name': 'has part, is conjugate base of', 'type': 'op_restriction', 'area_level': 2, 'parea_root_id': 'http://purl.obolibrary.org/obo/chebi_29067', 'label': '5-oxo-d-prolinate'})
2


In [ ]:
iri = 'http://purl.obolibrary.org/obo/chebi_103512' #False
# iri = 'http://purl.obolibrary.org/obo/chebi_29067' #True
# print(g.vs['name'])
# print(iri in g.vs['name'])

def checkInVertices(edgeEnd):
    return edgeEnd in g.vs['name']
    

def edgesPair(x):
    if checkInVertices(x.src) and checkInVertices(x.dst):
        return True
    else: 
        return False

# g2.edges.rdd.filter(lambda x: edgesPair(x)==True).count()
# g2.edges.rdd.filter(lambda x: edgesPair(x)==True).map(lambda x: (x.src,x.dst)).collect()
# print(g2.edges.rdd.filter(lambda x: edgesPair(x)==True).map(lambda x: (x.src,x.dst)).count())
g.add_edges(g2.edges.rdd.filter(lambda x: edgesPair(x)==True).map(lambda x: (x.src,x.dst)).collect())
g.es["ont"] = g2.edges.rdd.map(lambda x: x.ont).collect()
E=g.ecount()
print('total number of edges imported: ' , E)
print(g.es[0].tuple)
print(g.es[1].tuple)
print(g.es[101]['ont'])

In [81]:
iri = 'http://purl.obolibrary.org/obo/chebi_103512' #False
# iri = 'http://purl.obolibrary.org/obo/chebi_29067' #True
# print(g.vs['name'])
# print(iri in g.vs['name'])


# g2.edges.rdd.filter(lambda x: edgesPair(x)==True).count()
# g2.edges.rdd.filter(lambda x: edgesPair(x)==True).map(lambda x: (x.src,x.dst)).collect()
# print(g2.edges.rdd.filter(lambda x: edgesPair(x)==True).map(lambda x: (x.src,x.dst)).count())
g.add_edges(g2.edges.rdd.map(lambda x: (x.src,x.dst)).collect())
g.es["ont"] = g2.edges.rdd.map(lambda x: x.ont).collect()
E=g.ecount()
print('total number of edges imported: ' , E)
print(g.es[0].tuple)
print(g.es[1].tuple)
print(g.es[101]['ont'])

total number of edges imported:  2155
(1553, 1570)
(649, 1634)
chebi.obo


In [ ]:
spark.stop()

In [ ]:
sc.stop()